## CHATOLLAMA

Notebook creato per testare l'utilizzo "base" di ChatOllama sulla base della documentazione ufficiale

Visualizzo i modelli disponibili sull'istanza di Ollama "locale"

In [13]:
!ollama list

NAME                         ID              SIZE      MODIFIED    
llama3.2:3b-instruct-fp16    195a8c01d91e    6.4 GB    12 days ago    
llama3.2:latest              a80c4f17acd5    2.0 GB    12 days ago    
llama3.1:latest              42182419e950    4.7 GB    3 weeks ago    


Per avere un maggior dettaglio riguardo i modelli presenti su Ollama posso utilizzare la libreria Ollama

In [14]:
import ollama
ollama.list()

{'models': [{'name': 'llama3.2:3b-instruct-fp16',
   'model': 'llama3.2:3b-instruct-fp16',
   'modified_at': '2024-09-29T22:07:48.412703619+02:00',
   'size': 6433703586,
   'digest': '195a8c01d91ec3cb1e0aad4624a51f2602c51fa7d96110f8ab5a20c84081804d',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '3.2B',
    'quantization_level': 'F16'}},
  {'name': 'llama3.2:latest',
   'model': 'llama3.2:latest',
   'modified_at': '2024-09-29T17:42:43.594752215+02:00',
   'size': 2019393189,
   'digest': 'a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '3.2B',
    'quantization_level': 'Q4_K_M'}},
  {'name': 'llama3.1:latest',
   'model': 'llama3.1:latest',
   'modified_at': '2024-09-14T19:18:52.515326323+02:00',
   'size': 4661230766,
   'digest': '42182419e9508c30c4b1fe55

In [15]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

In [16]:
from langchain_core.messages import AIMessage
from langchain.callbacks.tracers import ConsoleCallbackHandler

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages, config={'callbaks' : [ConsoleCallbackHandler]})
ai_msg

AIMessage(content='Je aime le programmation.\n\n(Note: I used the informal "je" instead of the formal "on" because it\'s more suitable for casual conversations.)', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-12T10:50:15.004376373Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4917180401, 'load_duration': 1809238691, 'prompt_eval_count': 45, 'prompt_eval_duration': 816539000, 'eval_count': 32, 'eval_duration': 2203406000}, id='run-5e2f171c-6a63-407c-9072-3c2f19340390-0', usage_metadata={'input_tokens': 45, 'output_tokens': 32, 'total_tokens': 77})

In [17]:
print(ai_msg.content)

Je aime le programmation.

(Note: I used the informal "je" instead of the formal "on" because it's more suitable for casual conversations.)


Langchain, attraverso l'utilizzo di LCEL consente di concatenare, in una chain, il modello creato con ChatOllama con un PrompTemplate al fine di avere una maggiore possibilità di customizzazione

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Italian",
        "input": "I love programming in python.",
    }
)

AIMessage(content='Ti piace molto programmare con Python.\n\n(Translation note: I\'ve used the phrase "ti piace molto" which is a more informal way of saying "I really like" or "I\'m very fond of". If you\'d prefer a more formal translation, I can use "ti piace" instead.)\n\nVuoi parlare di un progetto specifico che stai lavorando con Python?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-12T10:50:53.423933769Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6377153542, 'load_duration': 18560456, 'prompt_eval_count': 42, 'prompt_eval_duration': 325451000, 'eval_count': 83, 'eval_duration': 5904416000}, id='run-ce1ff594-db51-4b26-ab25-71791bb939e7-0', usage_metadata={'input_tokens': 42, 'output_tokens': 83, 'total_tokens': 125})

PSe si utilizza un modello per il quale è stato fatto il fine-tune per renderlo compatibile con l'utilizzo dei tools è possibile richiamare funzioni dal LLM

In [11]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    model="llama3.2",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

[{'name': 'validate_user',
  'args': {'addresses': '["123 Fake St", "234 Pretend Boulevard"]',
   'user_id': '123'},
  'id': '9b7160f1-b280-4d44-b457-5ec22f2d65e0',
  'type': 'tool_call'}]